# Daniel Lee - FINAL Capstone Project

## i) Importing Libraries

In [3]:
#Import Library
import requests
from bs4 import BeautifulSoup 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random # library for random number generation

#Installing Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from mpl_toolkits.basemap import Basemap

import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans


# Matplotlib and numpy
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import numpy as np

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Folium installed and imported!')
print("Import complete!")

# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.

Folium installed and imported!
Import complete!


## ii) Setting up Foursquare Credentials

In [42]:
CLIENT_ID = 'O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT' # your Foursquare ID
CLIENT_SECRET = 'FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN' # your Foursquare Secret
VERSION = '20181101' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT
CLIENT_SECRET:FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN


## Part 1 - Exploratory Data Analysis using Foursquare

#### a) Explore the Number of gyms in SOHO *(within a 2km radius)
> *Represents Potential Demand of Sports or Exercise in Area*

In [206]:
latitude = 51.5136
longitude = -0.1365
print(latitude, longitude)
# 51.5054° N, 0.0235° W -- Canary Wharf
# 51.5136° N, 0.1365° W is SOHO London


venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map 

# add a red circle marker to represent the SOHO
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Gym',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

venues_map

51.5136 -0.1365


In [197]:
search_query = 'Gym'
radius = 2000
print("Gyms within 2 km of SOHO")

Gyms within 2 km of SOHO


In [208]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5136,-0.1365&v=20181101&query=Gym&radius=2000&limit=100'

In [209]:
results = requests.get(url).json()
# results

> Review the results

In [210]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
gyms = json_normalize(venues)
gyms.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
gyms.head()




,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,54c89520498e5d7951c9dd48,The May Fair Hotel,GB,Mayfair,United Kingdom,Stratton St,785,"[The May Fair Hotel (Stratton St), Mayfair, Gr...","[{'label': 'display', 'lat': 51.50848296133080...",51.508483,-0.144305,W1J 8LT,Greater London,The May Fair Gym,v-1546752302
1,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4bb222a3f964a52002bc3ce3,4-12 Regent St,GB,Piccadilly,United Kingdom,Rex House,602,"[4-12 Regent St (Rex House), Piccadilly, Great...","[{'label': 'display', 'lat': 51.50850603082001...",51.508506,-0.133562,SW1Y 4PE,Greater London,Pure Gym,v-1546752302
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4ae4991ff964a520259c21e3,9 Endell Street,GB,London,United Kingdom,NaN,874,"[9 Endell Street, London, Greater London, WC2H...","[{'label': 'display', 'lat': 51.51415625004893...",51.514156,-0.123908,WC2H 9SA,Greater London,Nuffield Health Fitness & Wellbeing Gym,v-1546752302
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5881e9efcc5b6a08df5a8768,31-36 Leicester Sq,GB,London,United Kingdom,NaN,613,"[31-36 Leicester Sq, London, Greater London, W...","[{'label': 'display', 'lat': 51.50995, 'lng': ...",51.509950,-0.129873,WC2H 7LH,Greater London,Radisson Blu Gym,v-1546752302
4,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,56fe22b0498ef2d2226413fb,NaN,GB,London,United Kingdom,NaN,531,"[London, Greater London, United Kingdom]","[{'label': 'display', 'lat': 51.51002945140006...",51.510029,-0.131398,NaN,Greater London,FIT (W Hotel Gym),v-1546752302


In [211]:
gyms.count()

categories                   50
hasPerk                      50
id                           50
location.address             38
location.cc                  50
location.city                44
location.country             50
location.crossStreet         10
location.distance            50
location.formattedAddress    50
location.labeledLatLngs      50
lat                          50
lng                          50
location.postalCode          33
location.state               44
name                         50
referralId                   50
dtype: int64

#### b) Explore the Number of Squash Centres in SOHO *(within a 5km radius)*
> *Represents level of Competition*

In [212]:
search_query2 = 'Squash'
radius2 = 5000
print("Squash within 5 km of SOHO")

Squash within 5 km of SOHO


In [213]:
url_Squash = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius2, LIMIT)
url_Squash

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5136,-0.1365&v=20181101&query=Squash&radius=5000&limit=100'

In [214]:
results2 = requests.get(url_Squash).json()
# results2



In [216]:
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe
Squash_df = json_normalize(venues2)
Squash_df.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
Squash_df.head()

print('{} Squash Centres were returned by Foursquare.'.format(Squash_df.shape[0]))


7 Squash Centres were returned by Foursquare.


In [217]:
Squash_df.head()
# Squash_df.shape[0]

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52e81612bcbc57f1066b7a26', 'name': 'R...",False,5321e2e1498ec1387434fb5b,NaN,GB,NaN,United Kingdom,NaN,1310,[United Kingdom],"[{'label': 'display', 'lat': 51.51530037022882...",51.515300,-0.117775,NaN,NaN,LSE Squash Courts,v-1546752313
1,"[{'id': '4bf58dd8d48988d1bd941735', 'name': 'S...",False,586f92c18cb34e38a5cb2802,46 Fish St Hill,GB,London,United Kingdom,NaN,3516,"[46 Fish St Hill, London, Greater London, EC3R...","[{'label': 'display', 'lat': 51.51046, 'lng': ...",51.510460,-0.085995,EC3R 6BR,Greater London,Olive + Squash,v-1546752313
2,"[{'id': '52e81612bcbc57f1066b7a2d', 'name': 'S...",False,571ccafc498e7a0a1b5de9f0,NaN,GB,NaN,United Kingdom,NaN,3224,[United Kingdom],"[{'label': 'display', 'lat': 51.52625092101996...",51.526251,-0.094622,NaN,NaN,Better Ironmonger Row Squash Courts,v-1546752313
3,"[{'id': '52e81612bcbc57f1066b7a2d', 'name': 'S...",False,59c65f17d8096e53406cedb5,NaN,GB,London,United Kingdom,NaN,3933,"[London, Greater London, SW8, United Kingdom]","[{'label': 'display', 'lat': 51.478542, 'lng':...",51.478542,-0.129367,SW8,Greater London,Southbank Club Squash Courts,v-1546752313
4,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4ac518f7f964a52092af20e3,Chalk Hill Road,GB,London,United Kingdom,NaN,5693,"[Chalk Hill Road, London, Greater London, W6 8...","[{'label': 'display', 'lat': 51.490529, 'lng':...",51.490529,-0.209826,W6 8DW,Greater London,Broadway Squash and Fitness Centre,v-1546752313


## PART 2 - Methodology 
### Linking London Areas to FourSquare Data

#### To identify the ideal area to start up a Squash Centre the following steps were conducted:
1. Calculate the number of gyms within a 2km radius and squash Centres within a 5km radius for each respective area 
2. Count the total number of gyms and squash Centres within the London Area
3. Create a column representing a weighting for 'Demand' and 'Competition' which will be: 
   * Demand = '# of Gyms / Total Number of Gyms'
   * Competition = '# of Squash Centres / Total Number of Squash Centres'
4. Create a last column, "Weighted_Demand" - adding the values from 'demand' and 'competition'
5. Rank the London Areas by Weighted_Demand in descending order

### Re-iterate for Canary Wharf & Shoreditch:
#### *Gyms within 2kms

> **Canary Wharf Gyms**

In [218]:
#Location for Canary Wharf - (51.5054° N, 0.0235° W)
latitude = 51.5054
longitude = -0.0235

LIMIT = 100
radius = 2000

print(latitude, longitude)

51.5054 -0.0235


In [219]:
url_gym2_CW = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_gym2_CW

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5054,-0.0235&v=20181101&query=Gym&radius=2000&limit=100'

In [220]:
results2_CW = requests.get(url_gym2_CW).json()

In [221]:
# assign relevant part of JSON to venues
venues_gym2_CW = results2_CW['response']['venues']

# tranform venues into a dataframe
gym2_CW = json_normalize(venues_gym2_CW)
gym2_CW.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
gym2_CW.head()



,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,52f9d5e6498e570fd759c768,NaN,GB,NaN,United Kingdom,NaN,390,[United Kingdom],"[{'label': 'display', 'lat': 51.50511784611449...",51.505118,-0.017881,NaN,NaN,NaN,HSBC Level 5 Gym,v-1546752339,NaN
1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5080019ce4b0db330e67d2eb,25 Bank Street,GB,London,United Kingdom,NaN,333,"[25 Bank Street, London, Greater London, E14 5...","[{'label': 'display', 'lat': 51.50333142475006...",51.503331,-0.020022,NaN,E14 5JP,Greater London,Nuffield Health Gym J.P. Morgan,v-1546752339,NaN
2,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,566ac90d498e25d72d60c351,"5 Hertsmere Road, West India Quay, Canary Wharf",GB,London,United Kingdom,NaN,273,"[5 Hertsmere Road, West India Quay, Canary Wha...","[{'label': 'display', 'lat': 51.50785446838057...",51.507854,-0.023271,NaN,E14 4AN,Greater London,PureGym,v-1546752339,NaN
3,"[{'id': '58daa1558bbb0b01f18ec203', 'name': 'O...",False,563f597acd10cb8941246557,NaN,GB,NaN,United Kingdom,NaN,770,[United Kingdom],"[{'label': 'display', 'lat': 51.50417496859146...",51.504175,-0.034442,NaN,NaN,NaN,TGO The Great Outdoor Gym,v-1546752339,NaN
4,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4b86753cf964a520ea8a31e3,8 Canada Square,GB,Canary Wharf,United Kingdom,Upper Bank Street,320,"[8 Canada Square (Upper Bank Street), Canary W...","[{'label': 'display', 'lat': 51.50442477905902...",51.504425,-0.019140,NaN,E14 5HQ,Greater London,Fifth Dimension Gym,v-1546752339,NaN


In [222]:
print('{} Gyms are near Canarywharf.'.format(gym2_CW.shape[0]))


49 Gyms are near Canarywharf.


> **Shoreditch Gyms**

In [130]:
#Location for Shoreditch  - (51.5285° N, 0.0847° W)

In [223]:
latitude = 51.5285
longitude = -0.0847
print(latitude, longitude)

51.5285 -0.0847


In [224]:
url_gym3_SHD = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_gym3_SHD

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5285,-0.0847&v=20181101&query=Gym&radius=2000&limit=100'

In [225]:
results3_SHD = requests.get(url_gym3_SHD).json()

In [226]:
# assign relevant part of JSON to venues
venues_gym3_SHD = results3_SHD['response']['venues']

# tranform venues into a dataframe
gym3_SHD = json_normalize(venues_gym3_SHD)
gym3_SHD.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
gym3_SHD.head()

print('{} Gyms are near Shoreditch.'.format(gym3_SHD.shape[0]))


50 Gyms are near Shoreditch.


### *Squash Centres within 5kms

> **Canary Wharf Squash Centres**

In [227]:
#Location for Canary Wharf - (51.5054° N, 0.0235° W)
latitude = 51.5054
longitude = -0.0235
LIMIT = 100
radius = 5000


print(latitude, longitude)

51.5054 -0.0235


In [228]:
url_Squash2_CW = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius2, LIMIT)
url_Squash2_CW

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5054,-0.0235&v=20181101&query=Squash&radius=5000&limit=100'

In [229]:
results_Sq2_CW = requests.get(url_Squash2_CW).json()


In [230]:
# assign relevant part of JSON to venues
venues_Sq2_CW = results_Sq2_CW['response']['venues']

# tranform venues into a dataframe
Sq2_CW = json_normalize(venues_Sq2_CW)
Sq2_CW.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
Sq2_CW.head()



,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1bd941735', 'name': 'S...",False,586f92c18cb34e38a5cb2802,46 Fish St Hill,GB,London,United Kingdom,4366,"[46 Fish St Hill, London, Greater London, EC3R...","[{'label': 'display', 'lat': 51.51046, 'lng': ...",51.51046,-0.085995,EC3R 6BR,Greater London,Olive + Squash,v-1546752369
1,"[{'id': '52e81612bcbc57f1066b7a2e', 'name': 'S...",False,59cf88cbf427de36738b10eb,"Address Blackheath Sports Club, Rectory Field,...",GB,London,United Kingdom,4702,"[Address Blackheath Sports Club, Rectory Field...","[{'label': 'display', 'lat': 51.47732, 'lng': ...",51.47732,0.027195,SE3,Greater London,Blackheath Squash Club,v-1546752369


In [231]:

print('{} Squash Centres are near Canary Wharf.'.format(Sq2_CW.shape[0]))

2 Squash Centres are near Canary Wharf.


> **Shoreditch Squash Centres**

In [232]:
#Location for Shoreditch  - (51.5285° N, 0.0847° W)
latitude = 51.5285
longitude = -0.0847
print(latitude, longitude)

51.5285 -0.0847


In [233]:
url_Squash2_SHD = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius2, LIMIT)
url_Squash2_SHD

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5285,-0.0847&v=20181101&query=Squash&radius=5000&limit=100'

In [234]:
results_Sq2_SHD = requests.get(url_Squash2_SHD).json()


In [235]:
# assign relevant part of JSON to venues
venues_Sq2_SHD = results_Sq2_SHD['response']['venues']

# tranform venues into a dataframe
Sq2_SHD = json_normalize(venues_Sq2_SHD)
Sq2_SHD.rename(columns={'location.lat':'lat', 'location.lng':'lng'}, inplace=True)
Sq2_SHD.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,lat,lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52e81612bcbc57f1066b7a2d', 'name': 'S...",False,571ccafc498e7a0a1b5de9f0,NaN,GB,NaN,United Kingdom,NaN,731,[United Kingdom],"[{'label': 'display', 'lat': 51.52625092101996...",51.526251,-0.094622,NaN,NaN,Better Ironmonger Row Squash Courts,v-1546752378
1,"[{'id': '4bf58dd8d48988d1bd941735', 'name': 'S...",False,586f92c18cb34e38a5cb2802,46 Fish St Hill,GB,London,United Kingdom,NaN,2010,"[46 Fish St Hill, London, Greater London, EC3R...","[{'label': 'display', 'lat': 51.51046, 'lng': ...",51.510460,-0.085995,EC3R 6BR,Greater London,Olive + Squash,v-1546752378
2,"[{'id': '52e81612bcbc57f1066b7a26', 'name': 'R...",False,5321e2e1498ec1387434fb5b,NaN,GB,NaN,United Kingdom,NaN,2721,[United Kingdom],"[{'label': 'display', 'lat': 51.51530037022882...",51.515300,-0.117775,NaN,NaN,LSE Squash Courts,v-1546752378
3,"[{'id': '4e39a956bd410d7aed40cbc3', 'name': 'T...",False,4d7a7cd813d5b60c572a7f9d,Courtside,GB,Crouch End,United Kingdom,Coolhurst Rd.,6244,"[Courtside (Coolhurst Rd.), Crouch End, Greate...","[{'label': 'display', 'lat': 51.57658668486048...",51.576587,-0.131151,N8 8EY,Greater London,Coolhurst Tennis & Squash Club,v-1546752378
4,"[{'id': '52e81612bcbc57f1066b7a2d', 'name': 'S...",False,59c65f17d8096e53406cedb5,NaN,GB,London,United Kingdom,NaN,6364,"[London, Greater London, SW8, United Kingdom]","[{'label': 'display', 'lat': 51.478542, 'lng':...",51.478542,-0.129367,SW8,Greater London,Southbank Club Squash Courts,v-1546752378


In [236]:
print('{} Squash Centres are near Shoreditch.'.format(Sq2_SHD.shape[0]))

6 Squash Centres are near Shoreditch.


## PART 3 - Mapping the Results

In [239]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[51.5285, -0.0847], zoom_start=12, tiles = "Stamen Toner")
# latitude = 51.5285
# longitude = -0.0847


# Blue Markers for Gym
    # SOHO
for lat, lng in zip(gyms.lat, gyms.lng):
    label = 'SOHO Gyms'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_opacity=0.7).add_to(map_london)  

    #Canary Wharf
for lat, lng in zip(gym2_CW.lat, gym2_CW.lng):
    label = 'Canary Wharf gyms'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='Green',
        fill=True,
        fill_opacity=0.7).add_to(map_london)

    #Shoreditch  
for lat, lng in zip(gym3_SHD.lat, gym3_SHD.lng):
    label = 'Shoreditch Gyms'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='purple',
        fill=True,
        fill_opacity=0.7).add_to(map_london)
     
        
# Red Markers for Squash Centres     

    # SOHO
for lat, lng in zip(Squash_df.lat, Squash_df.lng):
    label = 'SOHO Squash'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.7).add_to(map_london)    
    
    #Canary Wharf
for lat, lng in zip(Sq2_CW.lat, Sq2_CW.lng):
    label = 'Canary Wharf Squash'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='orange',
        fill=True,
        fill_opacity=0.7).add_to(map_london)  

    #Shoreditch
for lat, lng in zip(Sq2_SHD.lat, Sq2_SHD.lng):
    label = 'Shoreditch Squash'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='yellow',
        fill=True,
        fill_opacity=0.7).add_to(map_london)  
map_london

## RECOMMENDATION

The darker colours (Blue, Purple, Green) represent the gyms within the three areas for SOHO, Shoreditch and Canary Wharf respectively. 
If we can assume this is correlated to the demand of sports, there is a similar volume of gyms across the three areas.

On the otherhand, the brighter colours (red, yellow and orange) represents the competition within the three respective regions SOHO, Shoreditch and Canary Wharf.
we can see that there is a significant opportunity within the Canary Wharf region as there is a lack of squash courts near this area despite the consistently high volume of gyms in this area. 

**Conclusion** - As Canary wharf is one of the most densely populated areas by employees, the above insight poses a strong opportunity to be quite successful to open up a squash centre when only considering location as a factor. 